# Load and Save Files for CanRCM4 Downscaling Project
Load in and process HRDPS files here then export them as netcdf or numpy arrays

*This version is the 1st part of code review changes and comments by Doug of `LoadFiles.ipynb`.*

In [1]:
import datetime as dt
# import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path

#### DJL Comments on Cell Above

Be careful about aliasing imports to names that you might want to use as variable names like `dt`.
If, somewhere down in the notebook, you forget that you did `import datetime as dt` here,
and do something like `dt = t2 - t1` you will have re-assigned the value of `dt` and it could be 
very hard to figure out why your later use of `dt` to mean the `datatime` module isn't working.

The `pd`, `np`, and `xr` aliases are all pretty common, and not immediately obvious variables names
(though context matters!) so they are probably okay.
Personally, I rarely use import aliases because tab completion in Jupyter and my IDE mean that I don't have to
type more than a few letters, so why bother aliasing.

You are not using `pandas` anywhere in this notebook, so don't import it.
I realize that it is a carry over from you factoring this part of your processing pipeline out of
`proj-slp.ipynb`.

#### DJL Comments

I'm going to split the next cell into 2 cells to comment on it

In [2]:
#HRDPS data for all of 2016 to end of 2019
path = Path("/home/rbeutel/analysis/eosc510/proj.ipynb").resolve().parents[2]
path2 = "/results/forcing/atmospheric/GEM2.5/operational/"
path = path/path2

#### DJL Comments on Cell Above

I'm not sure what you are trying to do with `path` and `path2`.
The way `pathlib.Path` works means that, because `path2` starts with a `/`,
the `path = path/path2` statement will only assign the `path2` part to `path`.

I would write:

In [2]:
hrdps_path = Path("/results/forcing/atmospheric/GEM2.5/operational/")

#### DJL Comments on Cell Below
(because it has so many PerformanceWarnings)

Your glob expression is returning *a lot* more than the 2016-2019 files!
It is getting 2010-2019. The `*` means "any characters".
That probably explains why it was consuming so much memory on `salish`.
To get 2016-2019 you need to use `"ops_y201[6-9]*.nc"`. 
I was hoping that I could point you to a section in ch 7 of "The Linux Commandline" that 
would explain the way I'm using square brackets there, but it is not covered in that chapter.
Instead, you have to dive into ch 19 "Regular Expressions" and look at the 
"Bracket Expressions and Character Classes" section on pg 220.

For faster testing I would use glob expressions that select a few days, a month, or a few months; examples:
* `"ops_y2016m01d1[0-9]*.nc` will return 10 days in Jan 2016
* `"ops_y2016m03*.nc` will return Mar 2016
* `"ops_y2016m0[456]*.nc` or `"ops_y2016m0[4-6]*.nc` will return Apr-Jun 2016

You can test your glob expressions using `ls` in the shell.
You can even run a shell command from a notebook by putting a `!` in front of it:

In [10]:
!ls /results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d1[0-9]*.nc

/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d10.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d11.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d12.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d13.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d14.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d15.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d16.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d17.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d18.nc
/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d19.nc


In [3]:
hrdps = xr.open_mfdataset(sorted(hrdps_path.glob("ops_y201[6-9]*.nc")))

#### DJL Comments

The cell above is still slow (many minutes) and consumes a lot of memory (>100 Gb ???).
It is also the point where this process starts using [dask](https://docs.dask.org/) to
process files in parallel and manage the fact that 4 years of HRDPS are too large to fit
in memory.
`dask` handles that by splitting the processing up into a collection
(formally a graph) of tasks, and deferring execution of those tasks until the latest
possible time. In this case that is when you do the `np.concatenate()` operations below.

The challenge with `dask` (and parallel processing in general) is that it has to be tuned
for each particular workflow. All of my reading and experience of trying to use `dask` and
other parallel processing things has lead me to believe that there are few general rules for
how to do that tuning, mostly just guidelines.

So, I spent a bunch of interesting time yesterday afternoon trying to figure out how best to 
tune things for this next bit of processing. I'm not there yet, but I am making progress.
I decided to break this review here so that you could have the benefit of what is above
while I sort out the `dask` story.

To be continued...

In [5]:
#next we want the HRDPS data to be daily instead of hourly
day_avg = hrdps.resample(time_counter='D').mean(dim='time_counter')

In [7]:
#find index of extra day (leap day)
ind = list(day_avg.time_counter.values).index(np.datetime64('2016-02-29T00:00:00.000000000'))

#and then trim to be in a more relevant extent and remove leap day
#(dont need the HRDPS data to stretch as far as it does inland)
hrdps_ssc = day_avg.sel(x=slice(0., 480000.),time_counter=slice('2016-01-01 12:00:00', '2019-12-31 12:00:00'))

In [7]:
hrdps_P = np.concatenate((hrdps_ssc.atmpres.values[0:ind,:,:], hrdps_ssc.atmpres.values[(ind+1):,:,:]))
hrdps_U = np.concatenate((hrdps_ssc.u_wind.values[0:ind,:,:], hrdps_ssc.u_wind.values[(ind+1):,:,:]))
hrdps_V = np.concatenate((hrdps_ssc.v_wind.values[0:ind,:,:], hrdps_ssc.v_wind.values[(ind+1):,:,:]))
hrdps_T = np.concatenate((hrdps_ssc.tair.values[0:ind,:,:], hrdps_ssc.tair.values[(ind+1):,:,:]))

In [6]:
# set up titles
netcdf_title = 'HRDPSsubset.nc'
netcdf_comment = 'HRDPS2.5 datast used in CanRCM4 downscaling attempt for the Salish Sea'
notebook = 'LoadFiles.ipynb'

ds_attrs = {
        'creator_email':
            'rbeutel@eoas.ubc.ca',
        'institution_fullname': (
            'Earth, Ocean & Atmospheric Sciences,'
            ' University of British Columbia'
        ),
    'title': netcdf_title,
    'comment': netcdf_comment,
    'notebook': notebook,
    'summary': f'sea-level pressure, N/S wind, E/W wind, temperature',
    'history': (
            '[{}] File creation.'
            .format(dt.datetime.today().strftime('%Y-%m-%d'))
        )
}

coords_c = {
    'x' : np.arange(hrdps.x.shape[0]),
    'y' : np.arange(hrdps.y.shape[0]),
}

coords = {
    'x' : np.arange(hrdps.x.shape[0]),
    'y' : np.arange(hrdps.y.shape[0]),
    'time_counter' : hrdps.time_counter.values
}

data_c = {}
var_attrs_c = {} 

data_c['nav_lat'] = hrdps.nav_lat.values
var_attrs_c['nav_lat'] = {'units': 'degrees_north',
                       'long_name': 'latitude'}

data_c['nav_lon'] = hrdps.nav_lon.values
var_attrs_c['nav_lon'] = {'units': 'degrees_east',
                       'long_name': 'longitude'}

da = {}
for var in data_c:
    da[var] = xr.DataArray(
        data = data_c[var],
        name=var,
        dims=('y', 'x'),
        coords = coords_c,
        attrs = var_attrs_c[var])

data = {}
var_attrs = {}

var_attrs['slp'] = {'units': 'Pa',
                      'long_name': 'Pressure Reduced to MLS [Pa]'}
data['slp'] = hrdps_P

var_attrs['u_wind'] = {'units': 'm/s',
                      'long_name': 'U component of wind [m/s]'}
data['u_wind'] = hrdps_U

var_attrs['v_wind'] = {'units': 'm/s',
                      'long_name': 'V component of wind [m/s]'}
data['v_wind'] = hrdps_V

var_attrs['temp'] = {'units': 'oC',
                      'long_name': 'Surface temperature [oC]'}
data['temp'] = hrdps_T

for var in data:
    da[var] = xr.DataArray(
        data = data[var],
        name=var,
        dims=('time_counter', 'y', 'x'),
        coords = coords,
        attrs = var_attrs[var])
    

ds = xr.Dataset(
        data_vars={
            'nav_lat': da['nav_lat'],
            'nav_lon': da['nav_lon'],
            'slp': da['slp'],
            'u_wind': da['u_wind'],
            'v_wind': da['v_wind'],
            'temp': da['temp']},
        coords = coords,
        attrs = ds_attrs
)

ds

ValueError: different number of dimensions on data and dims: 3 vs 2